In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/kcv/Desktop/peak_power_test/stitched_detail_data/RD_LFP_PeakPower_REPT_TS_150_0001_0_100.csv')

In [4]:
df.columns

Index(['source_file', 'sheet_name', 'Record number', 'Sequence number of step',
       'Cycle No', 'Step No', 'Step name', 'Absolute time', 'Record time(m)',
       'Step time(h:m:s.ms)', 'volt(V)', 'Current(A)', 'Capacity(Ah)',
       'Energy(Wh)', 'Power(mW)', 'Internal R(Ω)', 'Charging energy(Wh)',
       'Discharge energy(Wh)', 'Charging capacity(Ah)',
       'Discharge capacity(Ah)', 'CUM charge E(Wh)', 'CUM discharge E(Wh)',
       'CUM charge C(Ah)', 'CUM discharge C(Ah)', 'Chamber T(℃)',
       'Chamber H(%)', 'Ripple F(Hz)', 'Ripple current(A)',
       'Total capacity(Ah)', 'DischargeVolt(V)', 'DischargeCurrent(A)',
       'DischargePower(mW)', 'water chiller outlet pressure(bar)',
       'water chiller return water pressure(bar)',
       'water chiller set flow(L/min)', 'water chiller set temperature(℃)',
       'Current water chiller temperature(℃)',
       'Current water chiller flow rate(L/min)', 'water chiller status',
       'water chiller mode', 'water chiller return wa

In [5]:
df[['Cycle No', 'Step name', 'Step No']].drop_duplicates()


,Cycle No,Step name,Step No
0,1,Rest,1
31,1,CC_Chg,2
403,1,Rest,3
464,1,CC_Chg,4
495,1,CC_Chg,5
579,1,Rest,6
640,1,CC_Chg,7
671,1,CC_Chg,8
745,1,Rest,9
806,1,CC_Chg,10


In [ ]:
import pandas as pd

# ensure Absolute time is datetime
df['Absolute time'] = pd.to_datetime(df['Absolute time'])

# take absolute values BEFORE grouping
df['Current_abs'] = df['Current(A)'].abs()
df['Capacity_abs'] = df['Capacity(Ah)'].abs()

# sort for correct sequencing
df = df.sort_values(['Cycle No',  'Absolute time','Step No'])

# group and aggregate
durations = (
    df.groupby(['Cycle No', 'Step name', 'Step No'])
      .agg(
          Start=('Absolute time', 'min'),
          End=('Absolute time', 'max'),

          Current_min=('Current_abs', 'min'),
          Current_max=('Current_abs', 'max'),

          Capacity_min=('Capacity_abs', 'min'),
          Capacity_max=('Capacity_abs', 'max'),

          Volt_min=('volt(V)', 'min'),
          Volt_max=('volt(V)', 'max')
      )
)

# compute duration in seconds
durations['Duration'] = (durations['End'] - durations['Start']).dt.total_seconds()

# create c-rate (Current_max / 150), rounded to 2 decimals
durations['c_rate'] = (durations['Current_max'] / 150).round(1)

# drop start & end if not needed
durations = durations.drop(columns=['Start', 'End'])

durations


Current_min  Current_max  Capacity_min  \
Cycle No Step name Step No                                           
1        CC_Chg    2          52.264988    52.269396           0.0   
                   4          74.995030    74.999281           0.0   
                   5          52.269226    52.274945           0.0   
                   7          74.994913    74.999284           0.0   
                   8          52.269002    52.275009           0.0   
                   10         74.994911    74.999323           0.0   
                   11         52.275007    52.275007           0.0   
                   13         74.995002    74.999321           0.0   
                   14         52.275068    52.275068           0.0   
                   16         74.995018    74.999578           0.0   
                   17         52.269061    52.274947           0.0   
                   19         74.995089    74.999603           0.0   
                   20         52.269227    52.274933           0.0   
                   22         74.995050    74.999376           0.0   
                   23         52.275068    52.275068           0.0   
                   25         74.994938    74.999319           0.0   
                   26         52.268863    52.275087           0.0   
                   28         74.995077    74.999317           0.0   
                   29         52.269107    52.274919           0.0   
         CC_DChg   32         74.994908    74.999055           0.0   
                   33         52.275072    52.275072           0.0   
                   35         74.994986    74.999147           0.0   
                   36         52.275100    52.275100           0.0   
                   38         74.994908    74.999165           0.0   
                   39         52.275035    52.275035           0.0   
                   41         74.994939    74.999118           0.0   
                   42         52.274958    52.274958           0.0   
                   44         74.994944    74.999077           0.0   
                   45         52.275041    52.275041           0.0   
                   47         74.994920    74.999070           0.0   
                   48         52.274951    52.274951           0.0   
                   50         74.995033    74.999068           0.0   
                   51         52.275091    52.275091           0.0   
                   53         74.995034    74.999077           0.0   
                   54         52.269200    52.275021           0.0   
                   56         74.995069    74.999043           0.0   
                   57         52.269158    52.274997           0.0   
                   59         52.264995    52.269453           0.0   
         Rest      1           0.000000     0.000000           0.0   
                   3           0.000000     0.000000           0.0   
                   6           0.000000     0.000000           0.0   
                   9           0.000000     0.000000           0.0   
                   12          0.000000     0.000000           0.0   
                   15          0.000000     0.000000           0.0   
                   18          0.000000     0.000000           0.0   
                   21          0.000000     0.000000           0.0   
                   24          0.000000     0.000000           0.0   
                   27          0.000000     0.000000           0.0   
                   30          0.000000     0.000000           0.0   
                   31          0.000000     0.000000           0.0   
                   34          0.000000     0.000000           0.0   
                   37          0.000000     0.000000           0.0   
                   40          0.000000     0.000000           0.0   
                   43          0.000000     0.000000           0.0   
                   46          0.000000     0.000000           0.0   
                   49          0.00000

In [11]:
dupes = df[df["Absolute time"].duplicated(keep=False)]
if dupes.empty:
    print("No duplicate timestamps.")
else:
    print(f"Found {dupes.shape[0]} rows sharing timestamps.")
    print(dupes.sort_values("Absolute time"))
    # or to see counts:
    print(df["Absolute time"].value_counts()[lambda s: s > 1])


Found 20 rows sharing timestamps.
                                        source_file    sheet_name  \
837    RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
838    RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
931    RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
932    RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
2299   RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
2300   RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
12700  RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
12701  RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
12795  RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
12794  RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
12888  RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
12889  RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
12982  RD_LFP_PeakPower_REPT_TS_150_0001_0_100.xlsx  Detail Data1   
